# ECE 396 - HHL Algorithm
#### Problem 1: Implement a 3x3 invertible but non-hermitian A in Qiskit

In [1]:
import numpy as np
from qiskit.algorithms.linear_solvers.hhl import HHL



We need to define a 3 x 3 invertible but non-hermitian A in Qiskit. That is, the following two qualities must hold:
$$AA^{-1} = A^{-1}A = I_n$$
$$A \neq A^{\dag}$$
If we restric ourselves to only real values, the matrix we can choose must then follow the two qualities: invertible and $A\neq A^T$.

One such matrix is the following:
$$A = \Bigg(\begin{matrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 1 & 1 


\end{matrix}\Bigg)$$


In [4]:
A = np.array([[1, 0, 0], [0, 1, 0], [0, 1, 1]])
print(A)

[[1 0 0]
 [0 1 0]
 [0 1 1]]


Before I can use the HHL algorithm, we need to convert the non-hermitian matrix into a hermitian matrix form. We can complete this by performing the operation defined in the article.